In [5]:
import sys
import os
import json
import re
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

os.environ['CUDA_VISIBLE_DEVICES']='1'
pd.set_option('display.max_colwidth', None)
sys.path.append('../../lib/')

In [6]:
import utility

### Training GPT:

2020-12-10 21:47:11.490665: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Arguments: Namespace(baseline=True, build_instance_version=None, dataset_cache='./jo_data_cache', dataset_path='/workspace/ceph_data/argument-undermining/data/gpt_data/jo_data.json', device='cuda', eval_before_start=False, fp16='', gradient_accumulation_steps=8, lm_coef=1.0, local_rank=-1, log_dir='/workspace/ceph_data/argument-undermining/models/gen_models/', lr=6.25e-05, max_history=2, max_norm=1.0, mc_coef=1.0, model_checkpoint='openai-gpt', n_epochs=6, num_candidates=2, output_model_checkpoint='jo_baseline_gpt', personality_permutations=1, premise_extra=False, train_batch_size=1, valid_batch_size=1)
INFO:/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/train.py:Prepare tokenizer, pretrained model and opt